<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Dataset" data-toc-modified-id="Dataset-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Dataset</a></span><ul class="toc-item"><li><span><a href="#MNIST" data-toc-modified-id="MNIST-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>MNIST</a></span></li><li><span><a href="#Split-train/val:-85/15%" data-toc-modified-id="Split-train/val:-85/15%-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Split train/val: 85/15%</a></span></li><li><span><a href="#Load" data-toc-modified-id="Load-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Load</a></span></li></ul></li></ul></div>

In [4]:
# our basic libraries
import torch
import torchvision

# data loading and transforming
from torch.utils.data import DataLoader
from torch.utils.data import random_split
from torchvision import transforms
from torchvision import datasets

# Dataset

## MNIST

In [20]:
transform = transforms.Compose([transforms.ToTensor(),
                               transforms.Normalize((0.1307,), (0.3081,))])

In [21]:
trainval_set = datasets.MNIST('./data', train=True, download=True,  # Downloads into a directory ../data
                               transform=transform)
test_set = datasets.MNIST('./data', train=False, download=False,  # No need to download again
                              transform=transform)

In [28]:
print(type(trainval_set[0][0]), type(trainval_set[0][1]))

<class 'torch.Tensor'> <class 'int'>


## Split train/val: 85/15%

In [33]:
trainval_size = len(trainval_set)
train_size, val_size = int(trainval_size*0.85), int(trainval_size*0.15)

# split
train_set, val_set = torch.utils.data.random_split(trainval_set, [train_size, val_size])

## Load

In [34]:
train_loader = DataLoader(train_set, batch_size=32, shuffle=True)
val_loader = DataLoader(val_set, batch_size=32, shuffle=True)
test_loader = DataLoader(test_set, batch_size=32, shuffle=True)